# Convolutional Neural Network Model (10-Fold-Cross-Validation)

This file contains 10-fold-cross-validation and parameter tuning in order to demonstrate the process of tunning the model.   
**For the final version of model and its accuracy and running time please see CNN-Trained.ipynb**

## Import Pakages

In [0]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import matplotlib.pyplot as plt
from imutils import build_montages
from keras import backend as K
from keras.utils import np_utils
from keras.datasets import fashion_mnist
from keras.optimizers import SGD, Adadelta
from sklearn.metrics import classification_report
import matplotlib
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from keras.wrappers.scikit_learn import KerasClassifier
import time
from keras.utils import plot_model
from keras.models import load_model
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold, cross_val_score

In [32]:
!cat /proc/cpuinfo




processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single pti ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf
bogomips	: 4600.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	: 1
vendor_id	: G

## Load and Process Data

In [0]:
# load data
((x_train, y_train), (x_test, y_test)) = fashion_mnist.load_data()
# rashape data
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))


In [0]:
# scale data to the range of [0, 1]
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

# initialize the label names
labelNames = ["t-Shirt", "trouser", "pullover", "dress", "coat", "sandal", "shirt", "sneaker", "bag", "ankle boot"]

## Specify Prameters

In [0]:
# number of epoches
NUM_EPOCHS = 30
# learning rate
LEARN_RATE = 0.1
# batch size
BATCH_SIZE = 200

# initialize the optimizer and model
adadelta=Adadelta()
sgd = SGD(lr=LEARN_RATE, momentum=0.9, decay=LEARN_RATE / NUM_EPOCHS)

## Construct Model

In [0]:
class CNN:
  def build(width=28, height=28, depth=1, optimizer=adadelta, dropout_rate = 0.25, BN = False):
    chanDim = -1
    input_shape = (height, width, depth)
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    if (BN is True):
      model.add(BatchNormalization(axis=chanDim))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    if (BN is True):
      model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_rate))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(10, activation='softmax'))
    
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

## Compile Model

In [0]:
# model = CNN.build(optimizer = adadelta, dropout_rate = 0, BN = False)

In [0]:
# model = CNN.build(optimizer = adadelta, dropout_rate = 0.25, BN = False)

In [0]:
# final model used
model = CNN.build(optimizer = adadelta, dropout_rate = 0.6, BN = False)

In [0]:
# model = CNN.build(optimizer = adadelta, dropout_rate = 0.6, BN = True)

In [0]:
# model = CNN.build(optimizer = sgd, dropout_rate = 0.6, BN = False)

In [0]:
# model = CNN.build(optimizer = adadelta, dropout_rate = 0.25, BN = True)

## Train Model

In [27]:
# define 10-fold cross validation test harness
seed = 7

kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

cvscores = []

start = time.time()

for i, (train_index, val_index) in enumerate(kf.split(x_train, y_train.argmax(1))):
  x_train_kf, x_val_kf = x_train[train_index], x_train[val_index]
  y_train_kf, y_val_kf = y_train[train_index], y_train[val_index]
  
  model = CNN.build(optimizer = adadelta, dropout_rate = 0.6, BN = False)
  model.fit(x_train_kf, y_train_kf, epochs=30, batch_size=200, verbose=1)

  scores = model.evaluate(x_val_kf, y_val_kf, verbose=0)
  print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

end = time.time()


Epoch 1/30
54000/54000 [==============================] - 4s 76us/step - loss: 0.7040 - acc: 0.7496
Epoch 2/30
54000/54000 [==============================] - 4s 67us/step - loss: 0.4474 - acc: 0.8405
Epoch 3/30
54000/54000 [==============================] - 4s 67us/step - loss: 0.3990 - acc: 0.8577
Epoch 4/30
54000/54000 [==============================] - 4s 68us/step - loss: 0.3652 - acc: 0.8711
Epoch 5/30
54000/54000 [==============================] - 4s 67us/step - loss: 0.3446 - acc: 0.8783
Epoch 6/30
54000/54000 [==============================] - 4s 68us/step - loss: 0.3246 - acc: 0.8852
Epoch 7/30
54000/54000 [==============================] - 4s 67us/step - loss: 0.3089 - acc: 0.8903
Epoch 8/30
54000/54000 [==============================] - 4s 68us/step - loss: 0.2953 - acc: 0.8951
Epoch 9/30
54000/54000 [==============================] - 4s 67us/step - loss: 0.2867 - acc: 0.8981
Epoch 10/30
54000/54000 [==============================] - 4s 67us/step - loss: 0.2749 - acc: 0.9026

## Genrate Result

In [0]:
# plot model structure
plot_model(model, to_file='model.png')

In [0]:
# make predictions on the test set
preds = model.predict(x_test)

# show a nicely formatted classification report
print(classification_report(y_test.argmax(axis=1), preds.argmax(axis=1),
                            target_names=labelNames))

N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy with Batch Normalization")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")


              precision    recall  f1-score   support

     t-Shirt       0.86      0.91      0.89      1000
     trouser       0.99      0.98      0.99      1000
    pullover       0.85      0.91      0.88      1000
       dress       0.92      0.94      0.93      1000
        coat       0.90      0.87      0.88      1000
      sandal       0.99      0.98      0.99      1000
       shirt       0.82      0.73      0.77      1000
     sneaker       0.96      0.99      0.97      1000
         bag       0.99      0.98      0.99      1000
  ankle boot       0.98      0.97      0.98      1000

    accuracy                           0.93     10000
   macro avg       0.93      0.93      0.93     10000
weighted avg       0.93      0.93      0.93     10000



/usr/local/lib/python3.6/dist-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [0]:
# save the model
model.save('CNN.h5')

In [0]:
# returns a compiled model
# identical to the previous one
model = load_model('CNN.h5')